In [78]:
import os #this module provides a portable way of using operating system dependent functionality
import pandas as pd #library providing high-performance, easy-to-use data structures and data analysis tools
import numpy as np #fundamental package for high-level mathematical functions
import xlrd #library for developers to extract data from Microsoft Excel spreadsheet files
from tqdm import tqdm # to show progress in some log loops
from IPython.core.display import HTML # to print some formatted HTML in jupyter notebook
import datetime
import ExcelExtraction

In [79]:
path = r"..\Raw\20200115"
count = 0
for root, dirs, files in os.walk(path):
    for file_ in files:
        count += 1
count

20

### Reading in all the files

In [80]:
def extract_all_files(path):
    """
    Keyword arguments:
    path = string

    goes through the folder structure of given path and collects all information and returns a list of pandas dataframe for further 
    processing
    """
    all_dfs = []

    for root, dirs, files in os.walk(path):
        
        for file_ in files:
            
            #skip if files not in xlsx, xls and csv file format 
            if not (file_.lower().endswith('xlsx') or file_.lower().endswith('xls') or file_.lower().endswith('csv') or file_.lower().endswith('xlsm') ):
                continue

            #skip any temp file
            if file_.lower().startswith('~$'):
                continue
            
            path_file = os.path.join(root, file_)


            #handling csv files
            if file_.endswith('csv'):
                df = pd.read_csv(path_file, error_bad_lines=False, encoding='ISO-8859-1')
                df['path'] = root
                df['file'] = file_
                df['sheet'] = 'csv'

                all_dfs.append(df)

            else:
                try:
                    #handling excel files
                    excel = pd.ExcelFile(path_file)
                    sheets = excel.sheet_names
                    for sheet in sheets:
                        if sheet!='Layout.':
                            continue
                        try:
                            df = excel.parse(sheet, skiprows=1)
                            df['path'] = root
                            df['file'] = file_
                            df['sheet'] = sheet
                            if len(df) > 0:
                                all_dfs.append(df)
                        except ValueError: 
                            print("value_error occured, check if there's any empty rows or columns in the beginning of the sheet")
                            print(path, file_, sheet)
                        except TypeError:
                        	print("type_error occured")
                        	print(path, file_, sheet)    



                except xlrd.XLRDError:
                    print('XLRDError on ', path_file, sheets)

    return all_dfs

In [81]:
all_files = extract_all_files(path)

In [82]:
len(all_files)

20

### Checking if 'Layout.' sheet has been picked or not

In [83]:
sheets = []
for key, df in enumerate(all_files):
    if df.sheet.values[0] not in sheets:
        sheets.append(df.sheet.values[0])
sheets

['Layout.']

In [ ]:
layout_dfs = []
for key, df in enumerate(all_files):
    if df.sheet.values[0]=='Layout.':
        layout_dfs.append(df.copy())

In [ ]:
len(layout_dfs)

In [84]:
file_sheet_keys = {}
for key, df in enumerate(all_files):
    file_sheet_keys[key] = (df.path.values[0], df.file.values[0], df.sheet.values[0])

### Collecting the primary data from top

In [85]:
ob_info_df = pd.DataFrame(columns=['Buyer_OB','Style_OB','Order_number_OB' ,'path', 'file', 'sheet'] )

for key, df in enumerate(all_files):
    buyer_ = df.loc[1, 'Unnamed: 1']
    style_ = df.loc[2, 'Unnamed: 1']
    order_ = df.loc[3, 'Unnamed: 1']
    ob_info_df.loc[len(ob_info_df)] = pd.Series( index = ['Buyer_OB','Style_OB','Order_number_OB', 'path', 'file', 'sheet'] ,
                                                  data=[buyer_, style_, order_, df.path[0], df.file[0], df.sheet[0]]) 

In [86]:
ob_info_df.count()

Buyer_OB           20
Style_OB           20
Order_number_OB     0
path               20
file               20
sheet              20
dtype: int64

In [87]:
ob_styles = ob_info_df.Style_OB.str.lower()

In [88]:
len(ob_styles)

20

### Collection the secondary data from next part

In [89]:
process_info_df = pd.DataFrame(columns=['Process_OB','Machine_type_OB','Individual_SMV_OB',
                                        'Standard_target_OB','Operator_OB','Helper_OB',
                                        'Actual_target_OB','path', 'file', 'sheet'])

for key, df in enumerate(all_files):
    
    try:
        start_row = df[df[df.columns[1]]=='Operation'].index[0]
    except IndexError:
        start_row = df[df[df.columns[0]]=='No.'].index[0]
        print(df.file[0])
          
    end_row   = df[df[df.columns[1]]=='TOTAL'].index[0]
    
    process_df = df.loc[start_row+2:end_row-1]
    
    process_df.rename(columns={
        
        'Unnamed: 1':'Process_OB',
        'Unnamed: 2':'Machine_type_OB',
        'Unnamed: 3':'Individual_SMV_OB',
        'Unnamed: 4':'Standard_target_OB',
        'Unnamed: 5':'Operator_OB',
        'Unnamed: 6':'Helper_OB', 
        'Unnamed: 7':'Actual_target_OB',
        
    }, inplace=True)
    process_df = process_df[process_info_df.columns]
    
    process_info_df = pd.concat([process_info_df, process_df], ignore_index=True)

In [90]:
process_df.head()

,Process_OB,Machine_type_OB,Individual_SMV_OB,Standard_target_OB,Operator_OB,Helper_OB,Actual_target_OB,path,file,sheet
11,Attach Shoulder (Both) - Chain Trim,O/L,0.303685,197.573,2,NaN,395.147,..\Raw\20200115,GUESS-K84I04 - SST with slv head gathering (n...,Layout.
12,"Attach Loop At Neck, 1 Loop With Fold",S/N,0.144,416.667,1,NaN,416.667,..\Raw\20200115,GUESS-K84I04 - SST with slv head gathering (n...,Layout.
13,"Attach Binding At Neck - Open, Auto Trim",F/L,0.163945,365.977,1,NaN,365.977,..\Raw\20200115,GUESS-K84I04 - SST with slv head gathering (n...,Layout.
14,Neck Binding End M & Cut,HEL,0.129,465.116,NaN,1,465.116,..\Raw\20200115,GUESS-K84I04 - SST with slv head gathering (n...,Layout.
15,Tack Inner & Outer At Neck Binding End,S/N,0.201,298.507,2,NaN,597.015,..\Raw\20200115,GUESS-K84I04 - SST with slv head gathering (n...,Layout.


In [91]:
process_info_df.count()

Process_OB            427
Machine_type_OB       415
Individual_SMV_OB     415
Standard_target_OB    415
Operator_OB           319
Helper_OB              97
Actual_target_OB      415
path                  447
file                  447
sheet                 447
dtype: int64

In [92]:
ob_df = pd.merge(ob_info_df, process_info_df, on=['path', 'file', 'sheet'])

In [93]:
len(ob_info_df), len(process_info_df), len(ob_df)

(20, 447, 447)

In [94]:
ob_df[ pd.isnull(ob_df.Process_OB) ].count()

Buyer_OB              20
Style_OB              20
Order_number_OB        0
path                  20
file                  20
sheet                 20
Process_OB             0
Machine_type_OB        0
Individual_SMV_OB      0
Standard_target_OB     0
Operator_OB            0
Helper_OB              0
Actual_target_OB       0
dtype: int64

In [95]:
ob_df = ob_df[pd.notnull(ob_df.Process_OB)]

In [96]:
ob_df.drop(['sheet'], axis=1, inplace=True)

In [97]:
len(ob_df)

427

In [98]:
ob_df.Operator_OB = ob_df.Operator_OB.replace(np.nan, 0)
ob_df.Helper_OB = ob_df.Helper_OB.replace(np.nan, 0)

In [99]:
ob_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 427 entries, 0 to 445
Data columns (total 12 columns):
Buyer_OB              427 non-null object
Style_OB              427 non-null object
Order_number_OB       0 non-null object
path                  427 non-null object
file                  427 non-null object
Process_OB            427 non-null object
Machine_type_OB       415 non-null object
Individual_SMV_OB     415 non-null object
Standard_target_OB    415 non-null object
Operator_OB           427 non-null float64
Helper_OB             427 non-null float64
Actual_target_OB      415 non-null object
dtypes: float64(2), object(10)
memory usage: 43.4+ KB


In [100]:
originalDF = ob_df.copy()

In [101]:
len(ob_df.drop_duplicates(['Style_OB', 'Process_OB'])), len(ob_df)

(375, 427)

In [102]:
ob_df.drop_duplicates(['Style_OB', 'Process_OB'],inplace=True)

In [103]:
ob_df.reset_index(drop=True,inplace=True)

In [104]:
ob_df['Style_OB'] = ob_df['Style_OB'].str.lower()

In [105]:
ob_df.path.value_counts()

..\Raw\20200115    375
Name: path, dtype: int64

In [106]:
ob_df.count()

Buyer_OB              375
Style_OB              375
Order_number_OB         0
path                  375
file                  375
Process_OB            375
Machine_type_OB       363
Individual_SMV_OB     363
Standard_target_OB    363
Operator_OB           375
Helper_OB             375
Actual_target_OB      363
dtype: int64

In [107]:
ob_df[pd.isnull(ob_df.Individual_SMV_OB)].count()

Buyer_OB              12
Style_OB              12
Order_number_OB        0
path                  12
file                  12
Process_OB            12
Machine_type_OB        0
Individual_SMV_OB      0
Standard_target_OB     0
Operator_OB           12
Helper_OB             12
Actual_target_OB       0
dtype: int64

In [108]:
ob_df[pd.isnull(ob_df.Individual_SMV_OB)]['Process_OB'].unique()

array(['Front Part', 'Back Part', 'Assembling', 'Back part', 'Front part',
       'Back Rise', 'Assembly'], dtype=object)

In [109]:
ob_df= ob_df[pd.notnull(ob_df.Individual_SMV_OB)]

In [110]:
ob_df.count()

Buyer_OB              363
Style_OB              363
Order_number_OB         0
path                  363
file                  363
Process_OB            363
Machine_type_OB       363
Individual_SMV_OB     363
Standard_target_OB    363
Operator_OB           363
Helper_OB             363
Actual_target_OB      363
dtype: int64

In [111]:
ob_df.columns

Index(['Buyer_OB', 'Style_OB', 'Order_number_OB', 'path', 'file', 'Process_OB',
       'Machine_type_OB', 'Individual_SMV_OB', 'Standard_target_OB',
       'Operator_OB', 'Helper_OB', 'Actual_target_OB'],
      dtype='object')

In [112]:
len(ob_df.groupby(by=["Buyer_OB",'Style_OB','path', 'file']))

17

In [113]:
len(ob_df.groupby(by=['Style_OB','path', 'file']))

17

In [114]:
for key, data in tqdm(ob_df.groupby(by=['Style_OB','path', 'file'])):
    ob_df.ix[data.index,"Total_SMV"] = data.Individual_SMV_OB.sum()
    ob_df.ix[data.index,"operation_id"] = pd.Series([i for i in range(len(data)+1) if i != 0],data.index)

  0%|                                                                                           | 0/17 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
100%|█████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 158.97it/s]


In [115]:
ob_df[pd.isnull(ob_df["operation_id"])]

,Buyer_OB,Style_OB,Order_number_OB,path,file,Process_OB,Machine_type_OB,Individual_SMV_OB,Standard_target_OB,Operator_OB,Helper_OB,Actual_target_OB,Total_SMV,operation_id


In [116]:
ob_df["factory_code"] = "1301"

In [117]:
ob_df.columns

Index(['Buyer_OB', 'Style_OB', 'Order_number_OB', 'path', 'file', 'Process_OB',
       'Machine_type_OB', 'Individual_SMV_OB', 'Standard_target_OB',
       'Operator_OB', 'Helper_OB', 'Actual_target_OB', 'Total_SMV',
       'operation_id', 'factory_code'],
      dtype='object')

In [118]:
ob_df.columns = [col.lower().replace(" ","_") for col in ob_df.columns if pd.notnull(col)]

In [119]:
ob_df.head(5)

,buyer_ob,style_ob,order_number_ob,path,file,process_ob,machine_type_ob,individual_smv_ob,standard_target_ob,operator_ob,helper_ob,actual_target_ob,total_smv,operation_id,factory_code
0,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,Attach Front Rise,O/L,0.189,317.46,1.0,0.0,317.46,4.491402,1.0,1301
1,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,Attach Back Rise,O/L,0.196993,304.58,1.0,0.0,304.58,4.491402,2.0,1301
2,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,"Set Front & Back Part, Together",HEL,0.147,408.163,0.0,1.0,408.163,4.491402,3.0,1301
3,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,"Seam Side - Bottom, Solid, No Label, Both Side...",O/L,0.319729,187.659,2.0,0.0,375.318,4.491402,4.0,1301
4,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,Inseam - Auto Trim,O/L,0.216118,277.627,2.0,0.0,555.253,4.491402,5.0,1301


## We wilL now deal with processes which has more than one manpower. 

In [120]:
ob_df.head()

,buyer_ob,style_ob,order_number_ob,path,file,process_ob,machine_type_ob,individual_smv_ob,standard_target_ob,operator_ob,helper_ob,actual_target_ob,total_smv,operation_id,factory_code
0,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,Attach Front Rise,O/L,0.189,317.46,1.0,0.0,317.46,4.491402,1.0,1301
1,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,Attach Back Rise,O/L,0.196993,304.58,1.0,0.0,304.58,4.491402,2.0,1301
2,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,"Set Front & Back Part, Together",HEL,0.147,408.163,0.0,1.0,408.163,4.491402,3.0,1301
3,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,"Seam Side - Bottom, Solid, No Label, Both Side...",O/L,0.319729,187.659,2.0,0.0,375.318,4.491402,4.0,1301
4,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,Inseam - Auto Trim,O/L,0.216118,277.627,2.0,0.0,555.253,4.491402,5.0,1301


In [121]:
bkupDF = ob_df.copy()

In [122]:
bkupDF['operator_ob'].unique()

array([1.  , 0.  , 2.  , 1.3 , 0.7 , 1.4 , 0.6 , 1.54, 0.46, 0.4 , 0.5 ,
       1.5 , 3.  , 4.  , 1.6 , 1.2 , 0.8 , 1.8 ])

In [123]:
bkupDF['helper_ob'].unique()

array([0. , 1. , 2. , 0.6, 0.5, 1.2, 0.8, 0.4, 1.5])

### Check if the manpower count seems unreal 

In [124]:
#bkupDF[bkupDF.helper_ob == 6]

In [125]:
bkupDF['allocated_mp'] = pd.Series([np.ceil(np.sum([i,j])) for i,j in zip(bkupDF['operator_ob'],bkupDF['helper_ob'])],index=bkupDF.index)

In [126]:
bkupDF['allocated_mp'].unique()

array([1., 2., 3., 4.])

In [127]:
len(bkupDF[bkupDF.allocated_mp == 0])

0

In [128]:
bkupDF.drop(bkupDF[bkupDF.allocated_mp == 0].index,inplace=True)
bkupDF.reset_index(drop=True,inplace=True)

In [129]:
bkupDF.head()

,buyer_ob,style_ob,order_number_ob,path,file,process_ob,machine_type_ob,individual_smv_ob,standard_target_ob,operator_ob,helper_ob,actual_target_ob,total_smv,operation_id,factory_code,allocated_mp
0,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,Attach Front Rise,O/L,0.189,317.46,1.0,0.0,317.46,4.491402,1.0,1301,1.0
1,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,Attach Back Rise,O/L,0.196993,304.58,1.0,0.0,304.58,4.491402,2.0,1301,1.0
2,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,"Set Front & Back Part, Together",HEL,0.147,408.163,0.0,1.0,408.163,4.491402,3.0,1301,1.0
3,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,"Seam Side - Bottom, Solid, No Label, Both Side...",O/L,0.319729,187.659,2.0,0.0,375.318,4.491402,4.0,1301,2.0
4,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,Inseam - Auto Trim,O/L,0.216118,277.627,2.0,0.0,555.253,4.491402,5.0,1301,2.0


In [130]:
letters = ['A','B','C','D','E','F','G','H',"I","J","K","L"]
DF = pd.DataFrame(columns=bkupDF.columns)
DF['op_no_ipa'] = np.nan
for row in tqdm(bkupDF.index):
    if int(bkupDF.ix[row,'allocated_mp'])>1:   
        for i in range(int(bkupDF.ix[row,'allocated_mp'])):
            values = bkupDF.ix[row]
            values["op_no_ipa"] = str(int(bkupDF.ix[row,"operation_id"]))+letters[i]
            DF.loc[len(DF)] = values
    else:
        values = bkupDF.ix[row]
        values["op_no_ipa"] = str(int(bkupDF.ix[row,"operation_id"]))
        DF.loc[len(DF)] = values

  0%|                                                                                          | 0/363 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.py

In [131]:
DF.head(15)

,buyer_ob,style_ob,order_number_ob,path,file,process_ob,machine_type_ob,individual_smv_ob,standard_target_ob,operator_ob,helper_ob,actual_target_ob,total_smv,operation_id,factory_code,allocated_mp,op_no_ipa
0,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,Attach Front Rise,O/L,0.189000,317.460317,1.0,0.0,317.460317,4.491402,1.0,1301,1.0,1
1,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,Attach Back Rise,O/L,0.196993,304.579969,1.0,0.0,304.579969,4.491402,2.0,1301,1.0,2
2,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,"Set Front & Back Part, Together",HEL,0.147000,408.163265,0.0,1.0,408.163265,4.491402,3.0,1301,1.0,3
3,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,"Seam Side - Bottom, Solid, No Label, Both Side...",O/L,0.319729,187.658806,2.0,0.0,375.317613,4.491402,4.0,1301,2.0,4A
4,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,"Seam Side - Bottom, Solid, No Label, Both Side...",O/L,0.319729,187.658806,2.0,0.0,375.317613,4.491402,4.0,1301,2.0,4B
5,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,Inseam - Auto Trim,O/L,0.216118,277.626626,2.0,0.0,555.253251,4.491402,5.0,1301,2.0,5A
6,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,Inseam - Auto Trim,O/L,0.216118,277.626626,2.0,0.0,555.253251,4.491402,5.0,1301,2.0,5B
7,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,Tack At Crotch Point,S/N,0.099000,606.060606,1.0,0.0,606.060606,4.491402,6.0,1301,1.0,6
8,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,Elastic M & Cut + Iron,HEL,0.120000,500.000000,0.0,1.0,500.000000,4.491402,7.0,1301,1.0,7
9,Guess,i01g07,NaN,..\Raw\20200115,GUESS-I01G07 - Kids Shorts with waistbelt & bo...,Make Elastic Ring,S/N,0.203000,295.566502,1.0,0.0,295.566502,4.491402,8.0,1301,1.0,8


In [132]:
DF.reset_index(drop=True,inplace=True)

In [133]:
DF.style_ob = DF.style_ob.str.strip()

In [134]:
cols = ['operation_id','factory_code','buyer_ob','style_ob','order_number_ob','process_ob','machine_type_ob',
        'individual_smv_ob','operator_ob','helper_ob','actual_target_ob','total_smv','op_no_ipa','standard_target_ob',
        'path', 'file']

In [135]:
DF = DF[cols]

In [136]:
col_dict = {'buyer_ob':"Buyer_OB",'style_ob':"Style_OB",'order_number_ob':"Order_number_OB",'process_ob':"Process_OB",
            'machine_type_ob':"Machine_type_OB",'individual_smv_ob':'Individual_SMV_OB','operator_ob':"Operator_OB",
            'helper_ob':'Helper_OB','actual_target_ob':"Actual_target_OB",'total_smv':"ttl_smv"}

In [137]:
DF.rename(columns=col_dict,inplace=True)

In [138]:
len(DF)

495

## Check if the styles have already been extracted, if not we need to append those with the master OB file. 

In [139]:
#reading in the main ob file
main_ob = pd.read_csv(r"../Done/MasterOB.csv")

In [140]:
len(main_ob)

73852

In [141]:
#checking if the column names are same
[item for item in DF.columns if item not in main_ob.columns.unique()]

[]

In [142]:
#checking if the styles are already in the main OB master data
style_list = [item for item in DF['Style_OB'].unique() if item in main_ob["Style_OB"].unique()]

In [143]:
style_list

['i01g07',
 'i74g03',
 'i74g06',
 'i74g19',
 'i74q11',
 'i82g10',
 'i82g21',
 'i93i07',
 'i94g01',
 'j73i49',
 'k84b07',
 'k84i01',
 'k84i04']

### This list should be empty, if not we need to look for the styles in the main database first (workbench ob table) and if they are there and have the same total SVM's, we will remove the files from the raw folder we are currently working on and extract again till the list is empty. But if the SVM's are different then we need to talk to the collection team to be sure whether they are new styles or not.

In [144]:
style_dict_db = {}
for key,data in DF[DF['Style_OB'].isin(style_list)].groupby(by='Style_OB'):
    style_dict_db[key] = float(str(data.ttl_smv.unique()[0])[:5])

In [145]:
style_dict_maindb = {}
for key,data in main_ob[main_ob['Style_OB'].isin(style_list)].groupby(by='Style_OB'):
    style_dict_maindb[key] = float(str(data.ttl_smv.unique()[0])[:5])

In [146]:
style_dict_db

{'i01g07': 4.491,
 'i74g03': 6.818,
 'i74g06': 6.305,
 'i74g19': 4.525,
 'i74q11': 6.337,
 'i82g10': 4.836,
 'i82g21': 6.912,
 'i93i07': 6.096,
 'i94g01': 6.143,
 'j73i49': 5.304,
 'k84b07': 3.175,
 'k84i01': 3.398,
 'k84i04': 3.502}

In [147]:
style_dict_maindb

{'i01g07': 5.475,
 'i74g03': 6.818,
 'i74g06': 6.883,
 'i74g19': 4.525,
 'i74q11': 6.337,
 'i82g10': 5.043,
 'i82g21': 6.912,
 'i93i07': 6.096,
 'i94g01': 5.189,
 'j73i49': 5.304,
 'k84b07': 3.175,
 'k84i01': 3.398,
 'k84i04': 3.502}

In [148]:
del_style_list = []
for style_ in style_list:
    if style_dict_db[style_] == style_dict_maindb[style_]:
        del_style_list.append(style_)
    else:
        new_style = style_+'-'+''.join(str(datetime.datetime.today().date()).split('-'))
        indices = DF[DF.Style_OB == style_].index
        DF.ix[indices,'Style_OB'] = new_style

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [149]:
del_style_list

['i74g03',
 'i74g19',
 'i74q11',
 'i82g21',
 'i93i07',
 'j73i49',
 'k84b07',
 'k84i01',
 'k84i04']

In [150]:
del_indices = DF[DF.Style_OB.isin(del_style_list)].index
del_path = originalDF[originalDF.Style_OB.str.lower().isin(del_style_list)].path.unique()[0]
del_files = originalDF[originalDF.Style_OB.str.lower().isin(del_style_list)].file.unique()

In [151]:
len(DF)

495

In [152]:
DF.drop(del_indices,inplace=True)
DF.reset_index(drop=True,inplace=True)

In [153]:
len(DF)

228

In [154]:
for file_ in del_files:
    os.remove(del_path+"\\"+file_)

### Updatting Master File

In [155]:
len(main_ob)

73852

In [156]:
#appending new file with the master file
main_ob = main_ob.append(DF, ignore_index=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [157]:
len(main_ob)

74080

### Making Output Files

In [161]:
today_ = ''.join(str(datetime.datetime.today().date()).split('-'))
DF.to_csv("../Done/1301 OB updated "+today_+".csv", index=False,encoding='utf-8')
DF.to_pickle("../Done/1301 OB updated "+today_)
main_ob.to_csv("../Done/MasterOB "+today_+".csv",index=False)

## Upload only the latest extracted file, not the main file (MasterOB) we just made.

# Creating the SQL Code for the server upload if required (optional)

In [ ]:
ob_df

In [ ]:
ob_df.columns

In [ ]:
for row in ob_df.index:
    print('INSERT INTO `operation_breakdown` (`operation_id`, `Buyer_OB`, `Style_OB`, `Order_number_OB`, `file`, `Process_OB`, `Machine_type_OB`, `Individual_SMV_OB`, `Standard_target_OB`, `Operator_OB`, `Helper_OB`, `Actual_target_OB`) VALUES (NULL,' 
    +"'"+str(ob_df.loc[row, 'Buyer_OB'])+"',"
    +"'"+str(ob_df.loc[row, 'Style_OB'])+"',"
    +"'"+str(ob_df.loc[row, 'Order_number_OB'])+"',"  
    +"'"+str(ob_df.loc[row, 'file'])+"',"
    +"'"+str(ob_df.loc[row, 'Process_OB'])+"',"
    +"'"+str(ob_df.loc[row, 'Machine_type_OB'])+"',"
    +"'"+str(ob_df.loc[row, 'Individual_SMV_OB'])+"',"
    +"'"+str(ob_df.loc[row, 'Standard_target_OB'])+"',"
    +"'"+str(ob_df.loc[row, 'Operator_OB'])+"',"
    +"'"+str(ob_df.loc[row, 'Helper_OB'])+"',"
    +"'"+str(ob_df.loc[row, 'Actual_target_OB'])+"');"
    )